# YOLOv8 Card Hand Classifier Training

This notebook trains a YOLOv8 image classification model to identify Clash Royale cards in your hand.

**Steps:**
1. Upload your dataset zip to Google Drive
2. Run all cells in order
3. Download the trained model (`best.pt`) at the end

**Expected dataset structure:**
```
card_hand_classifier.zip
└── card_hand_classifier/
    ├── train/
    │   ├── archers/
    │   ├── wizard/
    │   └── ...
    ├── val/
    │   └── ...
    └── test/
        └── ...
```

## 1. Setup Environment

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Install Ultralytics (YOLOv8)
!pip install ultralytics -q

## 2. Load Dataset

In [ ]:
# Unzip dataset from Google Drive
# CHANGE THIS PATH to match your Google Drive location
!unzip -q /content/drive/MyDrive/card_hand_classifier.zip -d /content/

# Verify dataset structure
!ls -l /content/card_hand_classifier/
!echo ""
!echo "Train classes:"
!ls /content/card_hand_classifier/train/ | head -10

## 3. Train Model

**Model Options:**
- `yolov8n-cls.pt` - Nano (fastest, ~15ms inference)
- `yolov8s-cls.pt` - Small (balanced, ~20ms inference) ⭐ Recommended
- `yolov8m-cls.pt` - Medium (most accurate, ~30ms inference)

In [ ]:
from ultralytics import YOLO

# Load pretrained model
model = YOLO('yolov8s-cls.pt')  # Small model (recommended)
# model = YOLO('yolov8n-cls.pt')  # Nano (fastest)
# model = YOLO('yolov8m-cls.pt')  # Medium (most accurate)

# Train
results = model.train(
    data='/content/card_hand_classifier',
    epochs=100,              # Number of training iterations
    imgsz=224,              # Image size (224 is standard)
    batch=32,               # Batch size (reduce to 16 if GPU memory issues)
    patience=15,            # Early stopping if no improvement
    save=True,              # Save best model
    project='card_classifier',
    name='run1',
    verbose=True,
    
    # Data augmentation (helps generalization)
    hsv_h=0.015,            # Hue variation
    hsv_s=0.7,              # Saturation variation
    hsv_v=0.4,              # Brightness variation
    degrees=5,              # Rotation (±5 degrees)
    translate=0.1,          # Translation
    scale=0.2,              # Scaling
    fliplr=0.5,             # 50% horizontal flip
)

## 4. Validate Model

In [ ]:
# Validate on test set
results = model.val()

print(f"\nTop-1 Accuracy: {results.top1:.2%}")
print(f"Top-5 Accuracy: {results.top5:.2%}")

## 5. View Training Results

In [ ]:
from IPython.display import Image, display

# View training curves
display(Image(filename='card_classifier/run1/results.png', width=800))

# View confusion matrix
display(Image(filename='card_classifier/run1/confusion_matrix_normalized.png', width=800))

## 6. Test on Sample Images

In [ ]:
import cv2
import matplotlib.pyplot as plt
from pathlib import Path
import random

# Load best model
best_model = YOLO('card_classifier/run1/weights/best.pt')

# Get random test images
test_dir = Path('/content/card_hand_classifier/test')
test_images = list(test_dir.glob('*/*.png')) + list(test_dir.glob('*/*.jpg'))
sample_images = random.sample(test_images, min(12, len(test_images)))

# Test and visualize
fig, axes = plt.subplots(3, 4, figsize=(15, 10))
axes = axes.flatten()

for idx, img_path in enumerate(sample_images):
    # Load image
    img = cv2.imread(str(img_path))
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Get true label from folder name
    true_label = img_path.parent.name
    
    # Predict
    results = best_model(img, verbose=False)
    probs = results[0].probs
    predicted = best_model.names[int(probs.top1)]
    confidence = float(probs.top1conf)
    
    # Visualize
    axes[idx].imshow(img_rgb)
    axes[idx].axis('off')
    
    # Color: green if correct, red if wrong
    color = 'green' if predicted == true_label else 'red'
    axes[idx].set_title(f'True: {true_label}\nPred: {predicted} ({confidence:.2f})', 
                        color=color, fontsize=10)

plt.tight_layout()
plt.show()

## 7. Export Model

Export to different formats for faster inference.

In [ ]:
# Export to ONNX for faster inference (optional)
best_model.export(format='onnx')

print("\n✅ Model exported!")
print("\nFiles to download:")
print("  1. card_classifier/run1/weights/best.pt  (PyTorch model)")
print("  2. card_classifier/run1/weights/best.onnx  (ONNX model - faster)")
print("  3. card_classifier/run1/results.png  (Training curves)")
print("  4. card_classifier/run1/confusion_matrix.png  (Confusion matrix)")

## 8. Download Model to Your Computer

Download the trained model and save it to `models/card_hand_classifier.pt` in your project.

In [ ]:
from google.colab import files

# Download best model
files.download('card_classifier/run1/weights/best.pt')

# Optionally download ONNX version (faster inference)
# files.download('card_classifier/run1/weights/best.onnx')

## Next Steps

1. **Download** the `best.pt` file
2. **Save** it to `models/card_hand_classifier.pt` in your project
3. **Update** `CardHandDetector` to use YOLOv8 instead of template matching
4. **Test** on real gameplay screenshots

See `TRAIN_CARD_CLASSIFIER.md` for integration instructions!